In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
import yfinance as yf

data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "SIGL",

        # ORIGINAL IS 60D but only for SIGL is 50D
        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "50d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "30m",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2021-01-11 09:30:00-05:00,7.490000,70.849998,7.200000,38.000000,283622
2021-01-11 10:00:00-05:00,33.160000,55.000000,22.000000,37.000000,515301
2021-01-11 10:30:00-05:00,37.000000,37.000000,26.010000,29.940001,239071
2021-01-11 11:00:00-05:00,29.940001,30.940001,24.049999,27.010000,223336
2021-01-11 11:30:00-05:00,27.000000,31.820000,27.000000,30.490000,138553
...,...,...,...,...,...
2021-03-23 12:00:00-04:00,2.430000,2.430000,2.430000,2.430000,1500
2021-03-23 12:30:00-04:00,2.450000,2.450000,2.450000,2.450000,199
2021-03-23 13:00:00-04:00,2.450000,2.450000,2.450000,2.450000,467


In [5]:
prices = data.reset_index()
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype                           
---  ------    --------------  -----                           
 0   Datetime  639 non-null    datetime64[ns, America/New_York]
 1   Open      639 non-null    float64                         
 2   High      639 non-null    float64                         
 3   Low       639 non-null    float64                         
 4   Close     639 non-null    float64                         
 5   Volume    639 non-null    int64                           
dtypes: datetime64[ns, America/New_York](1), float64(4), int64(1)
memory usage: 30.1 KB


In [6]:
prices['Datetime'] = prices['Datetime'].astype(str)

In [7]:
#remove -5:00
prices['Datetime'] = prices['Datetime'].str.replace('-05:00',"")
prices['Datetime'] = prices['Datetime'].str.replace('-04:00',"")

In [8]:
prices = prices[(prices['Datetime']>= '2020-12-22 09:30:00') & (prices['Datetime']<='2021-03-22 15:30:00')]
prices

,Datetime,Open,High,Low,Close,Volume
0,2021-01-11 09:30:00,7.490000,70.849998,7.200000,38.000000,283622
1,2021-01-11 10:00:00,33.160000,55.000000,22.000000,37.000000,515301
2,2021-01-11 10:30:00,37.000000,37.000000,26.010000,29.940001,239071
3,2021-01-11 11:00:00,29.940001,30.940001,24.049999,27.010000,223336
4,2021-01-11 11:30:00,27.000000,31.820000,27.000000,30.490000,138553
...,...,...,...,...,...,...
624,2021-03-22 13:00:00,2.600000,2.600000,2.600000,2.600000,216
625,2021-03-22 13:30:00,2.520000,2.520000,2.520000,2.520000,215
626,2021-03-22 14:30:00,2.550000,2.550000,2.550000,2.550000,1641
627,2021-03-22 15:00:00,2.600000,2.600000,2.600000,2.600000,134


In [9]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 629 entries, 0 to 628
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  629 non-null    object 
 1   Open      629 non-null    float64
 2   High      629 non-null    float64
 3   Low       629 non-null    float64
 4   Close     629 non-null    float64
 5   Volume    629 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 34.4+ KB


In [10]:
sentiment = pd.read_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/SIGL/60days_tweets.csv',lineterminator='\n')
sentiment.head()

,Datetime,Text,processed_tweet
0,2021-01-21 02:31:26+00:00,@NickytaLeb I agree markets can be irrational ...,i agree markets can be irrational amp traders ...
1,2021-01-21 05:00:44+00:00,@cperruna Isn’t that how the Greater Fool Theo...,isnt that how the greater fool theory has alwa...
2,2021-01-21 09:40:02+00:00,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl
3,2021-01-21 13:36:35+00:00,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...
4,2021-01-21 13:52:04+00:00,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...


In [11]:
sentiment = sentiment.rename(columns={'Datetime': 'created_at'})

In [12]:
#remove +00:00
sentiment['created_at'] = sentiment['created_at'].str.slice(0,19)

In [13]:
sentiment.head()

,created_at,Text,processed_tweet
0,2021-01-21 02:31:26,@NickytaLeb I agree markets can be irrational ...,i agree markets can be irrational amp traders ...
1,2021-01-21 05:00:44,@cperruna Isn’t that how the Greater Fool Theo...,isnt that how the greater fool theory has alwa...
2,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl
3,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...
4,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...


In [14]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   created_at       423 non-null    object
 1   Text             423 non-null    object
 2   processed_tweet  423 non-null    object
dtypes: object(3)
memory usage: 10.0+ KB


In [15]:
# convert datetime and created at to same datetime object
prices['Datetime'] = pd.to_datetime(prices['Datetime'])
sentiment['created_at'] = pd.to_datetime(sentiment['created_at'])

In [16]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 629 entries, 0 to 628
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  629 non-null    datetime64[ns]
 1   Open      629 non-null    float64       
 2   High      629 non-null    float64       
 3   Low       629 non-null    float64       
 4   Close     629 non-null    float64       
 5   Volume    629 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 34.4 KB


In [17]:
prices

,Datetime,Open,High,Low,Close,Volume
0,2021-01-11 09:30:00,7.490000,70.849998,7.200000,38.000000,283622
1,2021-01-11 10:00:00,33.160000,55.000000,22.000000,37.000000,515301
2,2021-01-11 10:30:00,37.000000,37.000000,26.010000,29.940001,239071
3,2021-01-11 11:00:00,29.940001,30.940001,24.049999,27.010000,223336
4,2021-01-11 11:30:00,27.000000,31.820000,27.000000,30.490000,138553
...,...,...,...,...,...,...
624,2021-03-22 13:00:00,2.600000,2.600000,2.600000,2.600000,216
625,2021-03-22 13:30:00,2.520000,2.520000,2.520000,2.520000,215
626,2021-03-22 14:30:00,2.550000,2.550000,2.550000,2.550000,1641
627,2021-03-22 15:00:00,2.600000,2.600000,2.600000,2.600000,134


In [18]:
# only close column is needed
prices = prices.drop(columns=['Open','High','Low','Volume'])

In [19]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   created_at       423 non-null    datetime64[ns]
 1   Text             423 non-null    object        
 2   processed_tweet  423 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 10.0+ KB


In [20]:
sentiment = sentiment.rename(columns={'created_at': 'Datetime'})
sentiment

,Datetime,Text,processed_tweet
0,2021-01-21 02:31:26,@NickytaLeb I agree markets can be irrational ...,i agree markets can be irrational amp traders ...
1,2021-01-21 05:00:44,@cperruna Isn’t that how the Greater Fool Theo...,isnt that how the greater fool theory has alwa...
2,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl
3,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...
4,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...
...,...,...,...
418,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four
419,2021-03-22 13:46:38,@dasmirts good one!,good one
420,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...
421,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...


In [21]:
# drop na of sentiment
sentiment = sentiment.dropna()

In [22]:
sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 422
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         423 non-null    datetime64[ns]
 1   Text             423 non-null    object        
 2   processed_tweet  423 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 13.2+ KB


In [23]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 629 entries, 0 to 628
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  629 non-null    datetime64[ns]
 1   Close     629 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 14.7 KB


In [24]:
df_merge = pd.merge_asof(sentiment,prices,left_on='Datetime',right_on="Datetime",direction='backward')
df_merge

,Datetime,Text,processed_tweet,Close
0,2021-01-21 02:31:26,@NickytaLeb I agree markets can be irrational ...,i agree markets can be irrational amp traders ...,7.634
1,2021-01-21 05:00:44,@cperruna Isn’t that how the Greater Fool Theo...,isnt that how the greater fool theory has alwa...,7.634
2,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.860
3,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.280
4,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.280
...,...,...,...,...
418,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.520
419,2021-03-22 13:46:38,@dasmirts good one!,good one,2.520
420,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.520
421,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.520


In [25]:
df_merge1 = df_merge[df_merge.Close.notnull()]
df_merge1

,Datetime,Text,processed_tweet,Close
0,2021-01-21 02:31:26,@NickytaLeb I agree markets can be irrational ...,i agree markets can be irrational amp traders ...,7.634
1,2021-01-21 05:00:44,@cperruna Isn’t that how the Greater Fool Theo...,isnt that how the greater fool theory has alwa...,7.634
2,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.860
3,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.280
4,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.280
...,...,...,...,...
418,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.520
419,2021-03-22 13:46:38,@dasmirts good one!,good one,2.520
420,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.520
421,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.520


In [26]:
import datetime

df_final = pd.DataFrame()

def workdays(d, end, excluded=(6, 7)):
    days = []
    while d.date() <= end.date():
        if d.isoweekday() not in excluded:
            days.append(d)
        d += datetime.timedelta(days=1)
    return days

total_days = workdays(datetime.datetime(2021, 1, 21), datetime.datetime(2021, 3, 23))
listofdates = []
for i in range (len(total_days)):
  date_time = total_days[i] + datetime.timedelta(minutes=30)
  date_time = date_time + datetime.timedelta(hours=15)
  end_time = date_time.strftime("%Y-%m-%d %H:%M:%S")

  date_time2 = total_days[i] + datetime.timedelta(hours=9)
  start_time = date_time2.strftime("%Y-%m-%d %H:%M:%S")
  
  listofdates.append([end_time, start_time])

# print(len(total_days))

for i in range(len(total_days)):
  df_final = df_final.append(df_merge1[(df_merge1['Datetime']<=listofdates[i][0]) & (df_merge1['Datetime']>=listofdates[i][1])])

In [27]:
df_final

,Datetime,Text,processed_tweet,Close
2,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.86
3,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.28
4,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.28
5,2021-01-21 14:48:47,"2 weeks after Musk's tweet, $SIGL is still tra...",weeks after musks tweet sigl is still trading ...,6.41
6,2021-01-21 14:57:40,@elonmusk what happen with sigl,what happen with sigl,6.41
...,...,...,...,...
418,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.52
419,2021-03-22 13:46:38,@dasmirts good one!,good one,2.52
420,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.52
421,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.52


In [28]:
df_final = df_final.reset_index(drop=True)
df_final

,Datetime,Text,processed_tweet,Close
0,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.86
1,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.28
2,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.28
3,2021-01-21 14:48:47,"2 weeks after Musk's tweet, $SIGL is still tra...",weeks after musks tweet sigl is still trading ...,6.41
4,2021-01-21 14:57:40,@elonmusk what happen with sigl,what happen with sigl,6.41
...,...,...,...,...
121,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.52
122,2021-03-22 13:46:38,@dasmirts good one!,good one,2.52
123,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.52
124,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.52


In [29]:
df_final["Comp"] = ''
df_final["Negative"] = ''
df_final["Neutral"] = ''
df_final["Positive"] = ''
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.86,,,,
1,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.28,,,,
2,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.28,,,,
3,2021-01-21 14:48:47,"2 weeks after Musk's tweet, $SIGL is still tra...",weeks after musks tweet sigl is still trading ...,6.41,,,,
4,2021-01-21 14:57:40,@elonmusk what happen with sigl,what happen with sigl,6.41,,,,
...,...,...,...,...,...,...,...,...
121,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.52,,,,
122,2021-03-22 13:46:38,@dasmirts good one!,good one,2.52,,,,
123,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.52,,,,
124,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.52,,,,


In [30]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         126 non-null    datetime64[ns]
 1   Text             126 non-null    object        
 2   processed_tweet  126 non-null    object        
 3   Close            126 non-null    float64       
 4   Comp             126 non-null    object        
 5   Negative         126 non-null    object        
 6   Neutral          126 non-null    object        
 7   Positive         126 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 8.0+ KB


In [31]:
df_final['processed_tweet'] = df_final['processed_tweet'].astype(str)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Datetime         126 non-null    datetime64[ns]
 1   Text             126 non-null    object        
 2   processed_tweet  126 non-null    object        
 3   Close            126 non-null    float64       
 4   Comp             126 non-null    object        
 5   Negative         126 non-null    object        
 6   Neutral          126 non-null    object        
 7   Positive         126 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 8.0+ KB


In [32]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
df_final['Comp'] = [analyzer.polarity_scores(x)['compound'] for x in df_final['processed_tweet']]
df_final['Negative'] = [analyzer.polarity_scores(x)['neg'] for x in df_final['processed_tweet']]
df_final['Neutral'] = [analyzer.polarity_scores(x)['neu'] for x in df_final['processed_tweet']]
df_final['Positive'] = [analyzer.polarity_scores(x)['pos'] for x in df_final['processed_tweet']]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [33]:
df_final

,Datetime,Text,processed_tweet,Close,Comp,Negative,Neutral,Positive
0,2021-01-21 09:40:02,@TheMaverickWS What's the price of these exper...,whats the price of these experts for sigl,6.86,0.0000,0.000,1.000,0.000
1,2021-01-21 13:36:35,$SIGL Massive Short Covering In Tech Shares SI...,sigl massive short covering in tech shares sig...,6.28,0.2960,0.000,0.845,0.155
2,2021-01-21 13:52:04,$SIGL Currently Reporting $0.1644 through the ...,sigl currently reporting through therd quarter...,6.28,0.0000,0.000,1.000,0.000
3,2021-01-21 14:48:47,"2 weeks after Musk's tweet, $SIGL is still tra...",weeks after musks tweet sigl is still trading ...,6.41,-0.2960,0.087,0.913,0.000
4,2021-01-21 14:57:40,@elonmusk what happen with sigl,what happen with sigl,6.41,0.0000,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...
121,2021-03-22 13:40:44,Lars von Trier's Fantastic Four,lars von triers fantastic four,2.52,0.5574,0.000,0.526,0.474
122,2021-03-22 13:46:38,@dasmirts good one!,good one,2.52,0.4404,0.000,0.256,0.744
123,2021-03-22 13:59:53,$GTII can someone who is smart put a countdown...,gtii can someone who is smart put a countdown ...,2.52,0.4019,0.000,0.938,0.062
124,2021-03-22 14:11:37,medium goals: going for a walk with @moshelink...,medium goals going for a walk with and ill b...,2.52,-0.4215,0.219,0.781,0.000


In [34]:
df_final = df_final.groupby(df_final.Datetime.dt.floor('30min')).mean()

In [35]:
df_final = df_final.drop(columns=['Negative','Neutral','Positive'])
df_final

,Close,Comp
Datetime,,
2021-01-21 09:30:00,6.86,0.000000
2021-01-21 13:30:00,6.28,0.148000
2021-01-21 14:30:00,6.41,-0.148000
2021-01-21 15:00:00,6.51,0.361200
2021-01-22 12:00:00,6.23,-0.117400
...,...,...
2021-03-19 10:00:00,2.67,0.000000
2021-03-19 13:30:00,2.53,0.000000
2021-03-22 12:30:00,2.56,0.361200


In [36]:
df_final.to_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/SIGL/final.csv')